In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
import matplotlib.pylab as plt
# https://docs.scipy.org/doc/numpy/reference/routines.math.html
import numpy as np

In [4]:
from datetime import tzinfo, timedelta, datetime

In [5]:
from distutils.version import StrictVersion

In [6]:
import sklearn

assert StrictVersion(sklearn.__version__ ) >= StrictVersion('0.18.1')

sklearn.__version__

'0.18.1'

In [7]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)

assert StrictVersion(tf.__version__) >= StrictVersion('1.1.0')

tf.__version__

'1.2.1'

In [8]:
import keras

assert StrictVersion(keras.__version__) >= StrictVersion('2.0.0')

keras.__version__

Using TensorFlow backend.


'2.0.6'

In [9]:
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/speed-limit-signs.zip
# !curl -O https://raw.githubusercontent.com/DJCordhose/speed-limit-signs/master/data/augmented-signs.zip    

In [10]:
# https://docs.python.org/3/library/zipfile.html
# from zipfile import ZipFile
# zip = ZipFile(r'speed-limit-signs.zip')
# zip.extractall('.')
# zip = ZipFile(r'augmented-signs.zip')
# zip.extractall('.')

In [11]:
# !ls -l speed-limit-signs

In [12]:
# !ls -l augmented-signs

In [13]:
import os
import skimage.data

def load_data(data_dir, type=".ppm"):
    """Loads a data set and returns two lists:
    
    images: a list of Numpy arrays, each representing an image.
    labels: a list of numbers that represent the images labels.
    """
    # Get all subdirectories of data_dir. Each represents a label.
    directories = [d for d in os.listdir(data_dir) 
                   if os.path.isdir(os.path.join(data_dir, d))]
    # Loop through the label directories and collect the data in
    # two lists, labels and images.
    labels = []
    images = []
    for d in directories:
        label_dir = os.path.join(data_dir, d)
        file_names = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith(type)]
        # For each label, load it's images and add them to the images list.
        # And add the label number (i.e. directory name) to the labels list.
        for f in file_names:
            images.append(skimage.data.imread(f))
            labels.append(int(d))
    return images, labels

In [14]:
# Load datasets.
ROOT_PATH = "./"
# data_dir = os.path.join(ROOT_PATH, "speed-limit-signs")
# images, labels = load_data(data_dir, type=".ppm")

data_dir = os.path.join(ROOT_PATH, "augmented-signs")
images, labels = load_data(data_dir, type=".png")

In [15]:
import skimage.transform

# Resize images
images64 = [skimage.transform.resize(image, (64, 64))
                for image in images]

In [16]:
import numpy as np

y = np.array(labels)
X = np.array(images64)
y.shape, X.shape

((3790,), (3790, 64, 64, 3))

In [17]:
from keras.utils.np_utils import to_categorical

num_categories = 6

y = to_categorical(y, num_categories)

In [18]:
from sklearn.model_selection import train_test_split

In [19]:
checkpoint_callback = keras.callbacks.ModelCheckpoint('../tmp/model-checkpoints/weights.epoch-{epoch:02d}-val_loss-{val_loss:.2f}.hdf5');

In [35]:
early_stopping_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1)

In [21]:
# https://github.com/tensorflow/tensorflow/blob/master/tensorflow/tensorboard/README.md
# https://keras.io/callbacks/#tensorboard
# http://stackoverflow.com/questions/42112260/how-do-i-use-the-tensorboard-callback-of-keras
tb_callback = keras.callbacks.TensorBoard(log_dir='../tmp/tf_log')
#                                          histogram_freq=1, write_graph=True, write_images=True)
#                                          histogram_freq=1, write_graph=True, write_images=True)
# tbCallBack = keras.callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)
# To start tensorboard
# tensorboard --logdir=/mnt/c/Users/olive/Development/ml/tf_log
# open http://localhost:6006

In [22]:
!rm -rf ../tmp/tf_log
!rm -rf ../tmp/model-checkpoints

!mkdir ../tmp/model-checkpoints
!mkdir ../tmp/tf_log

In [23]:
# training on a smaller subset due to lack of processing power, need to try on more powerful machine with all data

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.9, random_state=3)
# We can use all our data for training, because we have a completely separate set for testing later
X_train = X
y_train = y

In [24]:
X_train.shape, y_train.shape

((3790, 64, 64, 3), (3790, 6))

In [36]:
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Convolution2D, MaxPooling2D

# drop_out = 0.75
drop_out = 0.5
# drop_out = 0.25
# drop_out = 0.0

# input tensor for a 3-channel 64x64 image
inputs = Input(shape=(64, 64, 3))

# one block of convolutional layers
x = Convolution2D(64, 3, 3, activation='relu')(inputs)
x = Dropout(drop_out)(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = Dropout(drop_out)(x)
x = Convolution2D(64, 3, 3, activation='relu')(x)
x = Dropout(drop_out)(x)
x = MaxPooling2D(pool_size=(2, 2))(x)

# one more block
# x = Convolution2D(128, 3, 3, activation='relu')(x)
# x = Convolution2D(128, 3, 3, activation='relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Dropout(drop_out)(x)

# one more block
# x = Convolution2D(256, 3, 3, activation='relu')(x)
# x = MaxPooling2D(pool_size=(2, 2))(x)
# x = Dropout(drop_out)(x)

x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(drop_out)(x)

# softmax activation, 6 categories
predictions = Dense(6, activation='softmax')(x)
model = Model(input=inputs, output=predictions)
model.summary()
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 62, 62, 64)        1792      
_________________________________________________________________
dropout_15 (Dropout)         (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 60, 60, 64)        36928     
_________________________________________________________________
dropout_16 (Dropout)         (None, 60, 60, 64)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 58, 58, 64)        36928     
_________________________________________________________________
dropout_17 (Dropout)         (None, 58, 58, 64)        0         
__________

In [37]:
# Running on a GPU bach size might be critical depdendng on the GPU memory available
# more is desirable, but we might end up using 50 only 
print(datetime.utcnow().isoformat())
model.fit(X_train, y_train, epochs=150, batch_size=50, validation_split=0.3, 
          callbacks=[tb_callback, early_stopping_callback])
#           callbacks=[tb_callback])
# model.fit(X_train, y_train, epochs=50, batch_size=200, validation_split=0.3)
print(datetime.utcnow().isoformat())

2017-07-21T08:43:37.817327
Train on 2653 samples, validate on 1137 samples
Epoch 1/150
2653/2653 [==============================] - 7s - loss: 1.7734 - acc: 0.2929 - val_loss: 1.8906 - val_acc: 0.0000e+00
Epoch 2/150
2653/2653 [==============================] - 7s - loss: 1.3417 - acc: 0.3611 - val_loss: 3.3012 - val_acc: 0.0000e+00
Epoch 3/150
2653/2653 [==============================] - 7s - loss: 1.3424 - acc: 0.4078 - val_loss: 5.2970 - val_acc: 0.0000e+00
Epoch 4/150
2653/2653 [==============================] - 7s - loss: 1.2293 - acc: 0.4369 - val_loss: 6.7569 - val_acc: 0.0000e+00
Epoch 5/150
2653/2653 [==============================] - 7s - loss: 1.2265 - acc: 0.4538 - val_loss: 11.5121 - val_acc: 0.1381
Epoch 6/150
2653/2653 [==============================] - 7s - loss: 1.2597 - acc: 0.4828 - val_loss: 3.5925 - val_acc: 0.0343
Epoch 7/150
2653/2653 [==============================] - 7s - loss: 1.1286 - acc: 0.5168 - val_loss: 4.0041 - val_acc: 0.0079
Epoch 8/150
2653/2653 [===

### It really tries not to overfit, but in general we do not have enough training data

In [ ]:
train_loss, train_accuracy = model.evaluate(X_train, y_train, batch_size=32)
train_loss, train_accuracy

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=32)
test_loss, test_accuracy

In [ ]:
!mkdir models

In [ ]:
model.save('models/conv-vgg-augmented.hdf5')

In [ ]:
!ls -l models